<a href="https://atap.edu.au"><img src="https://www.atap.edu.au/atap-logo.png" width="125" height="50" align="right"></a>
# ATAP: TopSBM

TopSBM is a topic modelling approach that leverages a bipartite network of documents and terms and finding a hierarchy of blocks (or clusters) within the two types.

Australian Text Analytics Platform (ATAP) is an open source environment that provides researchers with tools and training for analysing, processing, and exploring text. ATAP: TopSBM is an effort to integrate the TopSBM approach developed by E.G. Altman et al which focuses on analysing and exploring your text.

**References**:
1. TopSBM: Topic Models based on Stochastic Block Models - https://topsbm.github.io/
2. ATAP: Australian Text Analytics Platform - https://www.atap.edu.au/
   

## Demo
This notebook is a demo of using TopSBM and integrates with `atap_corpus` from ATAP.

It first builds an ATAP Corpus and assign a 'title' as metadata and then compute a document-term-matrix DTM. <br>The DTM will be used as input to build the TopSBM network which the model will be fitted against. The 'title' metadata is used as label for the documents.

Then, wrap the model and corpus with the ATAP wrapper allows you to integrate your results with ATAP Corpus including at the end a download link for your Corpus with your results for re-use. You can then carry it across to another ATAP tool notebook available on our [website](https://www.atap.edu.au/) for further exploration or analysis.

In [ ]:
# todo: choice to upload your corpus.

In [3]:
import pandas as pd
# documents
with open('assets/corpus.txt', 'r', encoding='utf-8') as h:
    lines = h.readlines() 
df = pd.DataFrame(lines, columns=['document'])

# meta - title
with open('assets/titles.txt', 'r', encoding='utf-8') as h:
    titles = [l.rstrip() for l in h.readlines()]
df['title'] = titles
df.head(3)

,document,title
0,the nuclear overhauser effect noe is the tran...,Nuclear_Overhauser_effect Chemical_physics
1,a quantum solvent is essentially a superfluid...,Quantum_solvent Chemical_physics
2,coupling is a coupled rotational and vibratio...,Rovibrational_coupling Chemical_physics


In [15]:
from atap_corpus import Corpus

corpus = Corpus.from_dataframe(df, col_doc='document', name='topsbm')
f"Corpus <name: {corpus.name}   size: {len(corpus)} documents>"

'Corpus <name: topsbm   size: 63 documents>'

In [16]:
# create your document-term-matrix
dtm_name = 'tokens'                               # name of your DTM
tokeniser_func = lambda doc: doc.split()          # how you define each 'term' in the DTM from each document. (Here, it is whitespace delimited)

corpus.add_dtm_from_docs(tokeniser_func=tokeniser_func, name=dtm_name)
corpus.dtms[dtm_name]

<DTM 63 docs X 3140 terms>

## TopSBM: `make_graph()` and `fit()`

In [17]:
from topsbm.sbmtm import sbmtm

model = sbmtm()
model.make_graph(corpus.dtms['tokens'].to_lists_of_terms(), corpus['title'].tolist())
model.fit()

<NestedBlockState object, with base <BlockState object with 3203 blocks (63 nonempty), degree-corrected, for graph <Graph object, undirected, with 3203 vertices and 13050 edges, 2 internal vertex properties, 1 internal edge property, at 0x1c686b990>, at 0x1c6869310>, and 5 levels of sizes [(3203, 63), (63, 12), (12, 3), (3, 2), (2, 1)] at 0x1c5eb9890>


## Visualise the inferred hierarchical blocks

In [18]:
from atap_wrapper import visualise_hierarchy

vis_doc, vis_word = visualise_hierarchy(model, kind='collapsible-tree')

In [19]:
vis_doc

In [20]:
vis_word

## ATAP wrapper: Download 

Wrap your model and corpus with atap and then call serialise on the wrapper.

In [21]:
from atap_wrapper import wrap

wrapped = wrap(model, corpus, used_dtm=dtm_name) # dtm_name is the name of the dtm you've used earlier to build and fit the model.

In [22]:
wrapped.serialise("topsbm_corpus.zip")

'topsbm_corpus.zip'